In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()
data = spark.read.csv("SMSSpamCollection",inferSchema=True,sep='\t')
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

# from pyspark.sql import SQLContext
# from pyspark import SparkContext
# sc =SparkContext.getOrCreate()
# sqlContext = SQLContext(sc)
# data = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('../dataset/spam/train.csv')

In [2]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [6]:
from pyspark.sql.functions import length, regexp_replace

data = data.withColumn('length',length(data['text']))
data = data.drop('Subject')
data = data.withColumn("text", regexp_replace("text", ",", ""))\
    .withColumn("text", regexp_replace("text", ",", ""))\
    .withColumn("text", regexp_replace("text", ":", ""))\
    .withColumn("text", regexp_replace("text", "-", ""))\
    .withColumn("text", regexp_replace("text", "/", ""))\
    .withColumn("text", regexp_replace("text", "\n", ""))\
    .withColumn("text", regexp_replace("text", "  ", ""))\
    .withColumn("text", regexp_replace("text", ";", ""))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [7]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [8]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [9]:
from pyspark.ml.classification import NaiveBayes
# Use defaults
nb = NaiveBayes()

In [10]:
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)

In [11]:
clean_data = clean_data.select(['label','features'])
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(12955,[5,10,30,6...|
|  0.0|(12955,[0,20,301,...|
|  1.0|(12955,[2,9,17,23...|
|  0.0|(12955,[0,62,81,1...|
|  0.0|(12955,[36,157,32...|
|  1.0|(12955,[11,14,59,...|
|  0.0|(12955,[11,54,108...|
|  0.0|(12955,[125,177,4...|
|  1.0|(12955,[1,46,109,...|
|  1.0|(12955,[0,1,9,26,...|
|  0.0|(12955,[16,43,121...|
|  1.0|(12955,[6,14,35,9...|
|  1.0|(12955,[9,23,46,8...|
|  0.0|(12955,[39,95,202...|
|  0.0|(12955,[552,1687,...|
|  1.0|(12955,[23,97,123...|
|  0.0|(12955,[75,213,36...|
|  0.0|(12955,[0,2,49,99...|
|  0.0|(12955,[0,72,100,...|
|  1.0|(12955,[3,23,78,1...|
+-----+--------------------+
only showing top 20 rows



In [37]:
(training,testing) = clean_data.randomSplit([0.7,0.3])
spam_predictor = nb.fit(training)
test_results = spam_predictor.transform(testing)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,3,9,1...|[-576.04809574027...|[0.99999999999307...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-846.76807909555...|[1.0,1.1615186239...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1366.6474191483...|[1.0,1.9203829937...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-217.12938114444...|[1.0,8.0460229588...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-689.09237209870...|[1.0,1.3726090163...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-972.28920896551...|[1.0,3.1382747324...|       0.0|
|  0.0|(13424,[0,1,24,31...|[-355.29440607134...|[1.0,1.3454933541...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-1485.6805906997...|[0.99999969182573...|       0.0|
|  0.0|(13424,[0,1,27,88...|[-1521.6341948175...|[0.02485519397279...|       1.0|
|  0.0|(13424,[0

In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9247526392130049
